In [1]:
import tkinter as tk
from tkinter import ttk
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import requests
import joblib

# Load the trained model and scaler (assuming they are saved as 'model.pkl' and 'scaler.pkl')
scaler = joblib.load('scaler.pkl')
model = joblib.load('weather_prediction_model.joblib')

# API key for OpenWeatherMap
API_KEY = "836d3749c442112d8522b8596ee2d52a"

def fetch_weather_data(location):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': location,
        'appid': API_KEY,
        'units': 'metric'
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    if response.status_code != 200:
        return None

    weather_data = {
        "Present_Tmax": data['main']['temp_max'],
        "Present_Tmin": data['main']['temp_min'],
        "LDAPS_RHmin": data['main']['humidity'],  # Approximation
        "LDAPS_RHmax": data['main']['humidity'],  # Approximation
        "LDAPS_Tmax_lapse": data['main']['temp_max'],  # Approximation
        "LDAPS_Tmin_lapse": data['main']['temp_min'],  # Approximation
        "LDAPS_WS": data['wind']['speed'],
        "LDAPS_LH": 0,  # Placeholder, not available from OpenWeatherMap
        "LDAPS_CC1": data['clouds']['all'],  # Approximation
        "LDAPS_CC2": data['clouds']['all'],  # Approximation
        "LDAPS_CC3": data['clouds']['all'],  # Approximation
        "LDAPS_CC4": data['clouds']['all'],  # Approximation
        "LDAPS_PPT1": 0,  # Placeholder, not available from OpenWeatherMap
        "LDAPS_PPT2": 0,  # Placeholder, not available from OpenWeatherMap
        "LDAPS_PPT3": 0,  # Placeholder, not available from OpenWeatherMap
        "LDAPS_PPT4": 0,  # Placeholder, not available from OpenWeatherMap
        "lat": data['coord']['lat'],
        "lon": data['coord']['lon'],
        "DEM": 0,  # Placeholder, not available from OpenWeatherMap
        "Slope": 0,  # Placeholder, not available from OpenWeatherMap
        "Solar radiation": 0  # Placeholder, not available from OpenWeatherMap
    }
    return weather_data

# Create the main application window
root = tk.Tk()
root.title("Extreme Weather Prediction App")
root.geometry("1500x800")  # Increased window size

# Set the background color of the root window
root.configure(bg='teal')

# Create a frame for the input fields and labels
frame = tk.Frame(root, bg='teal')
frame.place(relwidth=0.8, relheight=0.9, relx=0.1, rely=0.05)

# Add a heading label
heading_label = tk.Label(frame, text="Extreme Weather Prediction App", font=("Helvetica", 20), bg='teal', fg='white')
heading_label.grid(row=0, columnspan=4, pady=20)

# Create and place the location entry
location_label = tk.Label(frame, text="Location", font=("Helvetica", 12), bg='teal', fg='white')
location_label.grid(row=1, column=0, pady=5, padx=10, sticky='e')
location_entry = tk.Entry(frame, font=("Helvetica", 12))
location_entry.grid(row=1, column=1, pady=5, padx=10, sticky='w')


# Create and place labels and entry fields for each input variable
input_labels = [
    "Present_Tmax", "Present_Tmin", "LDAPS_RHmin", "LDAPS_RHmax",
    "LDAPS_Tmax_lapse", "LDAPS_Tmin_lapse", "LDAPS_WS", "LDAPS_LH",
    "LDAPS_CC1", "LDAPS_CC2", "LDAPS_CC3", "LDAPS_CC4",
    "LDAPS_PPT1", "LDAPS_PPT2", "LDAPS_PPT3", "LDAPS_PPT4",
    "lat", "lon", "DEM", "Slope", "Solar radiation"
]

entries = {}
for i, label in enumerate(input_labels):
    lbl = tk.Label(frame, text=label, font=("Helvetica", 12), bg='teal', fg='white')
    ent = tk.Entry(frame, font=("Helvetica", 12))
    row = (i % (len(input_labels) // 2)) + 2
    col = (i // (len(input_labels) // 2)) * 2
    lbl.grid(row=row, column=col, pady=5, padx=10, sticky='e')
    ent.grid(row=row, column=col + 1, pady=5, padx=10, sticky='w')
    entries[label] = ent

# Function to fetch weather data and fill the entries
def autofill():
    location = location_entry.get()
    weather_data = fetch_weather_data(location)

    if weather_data:
        for label, value in weather_data.items():
            entries[label].delete(0, tk.END)
            entries[label].insert(0, value)
    else:
        result_label.config(text="Error fetching data. Please check the location name.")

# Create and place the Auto-Fill button beside the location entry
autofill_button = tk.Button(frame, text="Auto-Fill", command=autofill, font=("Helvetica", 14))
autofill_button.grid(row=1, column=2, pady=5, padx=10, sticky='w')

# Function to make a prediction
def predict():
    try:
        # Collect input values
        input_values = [float(entries[label].get()) for label in input_labels]

        # Scale the input values
        input_values = np.array(input_values).reshape(1, -1)
        scaled_values = scaler.transform(input_values)

        # Make a prediction
        prediction = model.predict(scaled_values)

        # Display the prediction
        result_label.config(text=f"Predicted Next_Tmax: {prediction[0]:.2f} °C")
    except ValueError:
        result_label.config(text="Please enter valid numerical values.")

# Create and place the Predict button
predict_button = tk.Button(frame, text="Predict", command=predict, font=("Helvetica", 14))
predict_button.grid(row=(len(input_labels) // 2) + 2, columnspan=4, pady=20)

# Create and place a label to display the prediction result
result_label = tk.Label(frame, text="Predicted Next_Tmax: N/A", font=("Helvetica", 16), bg='teal', fg='white')
result_label.grid(row=(len(input_labels) // 2) + 3, columnspan=4, pady=20)

# Run the application
root.mainloop()
